In [1]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Sat Oct 21 09:54:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8     7W /  N/A |     99MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [5]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]]) 
    random.shuffle(sequential_data)
    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle (buys)
    random.shuffle (sells)
    # We want to know what is the minimum vlaue of this two lists.
    lower = min(len(buys), len(sells))
    # Upto 30K Buys
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)

    # We will do X, y for model.fit(X, y)
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]
#preprocess_df (main_df)
train_x, train_y = preprocess_df (main_df)
validation_x, validation_y = preprocess_df (validation_main_df)
print(f"train_x.shape: {train_x.shape}, len(train_y): {len(train_y)}")
print(f"Train Data Size: {len(train_x)}, Validation Data Size: {len(validation_x)}")
print(f"Train Data:")
print(f"  Buys List: {train_y.count(1)}")
print(f"  Don't Buy List: {train_y.count(0)}")
print(f"Validation Data:")
print(f"  Buys List: {validation_y.count(1)}")
print(f"  Don't Buys List: {validation_y.count(0)}")

train_x.shape: (69188, 60, 8), len(train_y): 69188
Train Data Size: 69188, Validation Data Size: 3062
Train Data:
  Buys List: 34594
  Don't Buy List: 34594
Validation Data:
  Buys List: 1531
  Don't Buys List: 1531


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
# https://stackoverflow.com/questions/47857437/keras-importerror-cannot-import-name-cudnnlstm
# tensorflow LSTM already has CuDNNLSTM
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"
# Deep Learning: Epoch = One Pass (Forward + Backward)
# Reinforcement Learning: Episode = Terminnal state (State, Reward)
EPOCHS = 10
BATCH_SIZE = 64     # 64 rows of dataset
# Uinque Model Name: Sequence_Length-Sequence_Period-Time_Stamp
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]]) 
    random.shuffle(sequential_data)
    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle (buys)
    random.shuffle (sells)
    # We want to know what is the minimum vlaue of this two lists.
    lower = min(len(buys), len(sells))
    # Upto 30K Buys
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)

    # We will do X, y for model.fit(X, y)
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]
#preprocess_df (main_df)
train_x, train_y = preprocess_df (main_df)
validation_x, validation_y = preprocess_df (validation_main_df)
print(f"Train Data Size: {len(train_x)}, Validation Data Size: {len(validation_x)}")
print(f"Train Data:")
print(f"  Buys List: {train_y.count(1)}")
print(f"  Don't Buy List: {train_y.count(0)}")
print(f"Validation Data:")
print(f"  Buys List: {validation_y.count(1)}")
print(f"  Don't Buys List: {validation_y.count(0)}")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Both tensorflow LSTM and NVIDIA cudaDNN have the GPU LSTM cell.
# NIVIDIA cudaDNN even have more optimized LSTM cell with 5 x faster. 
from tensorflow.keras.layers import Dense, Dropout, LSTM
mnist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()
print('train_x.shape:', train_x.shape)
print('train_x.shape[1:]:', train_x.shape[1:])
# train_x.shape: (69188, 60, 8)
# train_x.shape[1:]: (60, 8)
x_train = train_x/250.0
x_test = test_x/255.0

model = Sequential ()
# we add the 128 LSTM cell, 
print('x_train.shape[1:]:', x_train.shape[1:])
# x_train.shape[1:] = (28, 28)
# We can do cudaNNLSTM later on.
# model.add(cudaNNLSTM(128, input_shape=(x_train.shape[1:]), activation='relu'))
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences = True))
model.add(Dropout(0.2)) # we dropout 20% neurons.
# reurn_sequences = True is only use to the output layer.
# If go the another layer, we don't need return_sequences

model.add(LSTM(128, activation='relu')) # hidden layer does not need the input_shape
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))  # Make 32 nodes in the dense layer.
model.add(Dropout(0.2))

2023-10-23 17:22:59.271626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 17:22:59.271655: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 17:22:59.273031: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 17:22:59.421892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 17:23:00.584412: W tensorflow/compiler/

Train Data Size: 69188, Validation Data Size: 3062
Train Data:
  Buys List: 34594
  Don't Buy List: 34594
Validation Data:
  Buys List: 1531
  Don't Buys List: 1531
train_x.shape: (60000, 28, 28)
train_x.shape[1:]: (28, 28)
x_train.shape[1:]: (28, 28)


2023-10-23 17:23:05.933536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-23 17:23:05.933566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: knode1
2023-10-23 17:23:05.933571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: knode1
2023-10-23 17:23:05.933670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.199.2
2023-10-23 17:23:05.933687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.199.2
2023-10-23 17:23:05.933691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.199.2


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
# https://stackoverflow.com/questions/47857437/keras-importerror-cannot-import-name-cudnnlstm
# tensorflow LSTM already has CuDNNLSTM
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"
# Deep Learning: Epoch = One Pass (Forward + Backward)
# Reinforcement Learning: Episode = Terminnal state (State, Reward)
EPOCHS = 10
BATCH_SIZE = 64     # 64 rows of dataset
# Uinque Model Name: Sequence_Length-Sequence_Period-Time_Stamp
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]]) 
    random.shuffle(sequential_data)
    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle (buys)
    random.shuffle (sells)
    # We want to know what is the minimum vlaue of this two lists.
    lower = min(len(buys), len(sells))
    # Upto 30K Buys
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)

    # We will do X, y for model.fit(X, y)
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y



main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)


main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]




#preprocess_df (main_df)
train_x, train_y = preprocess_df (main_df)
validation_x, validation_y = preprocess_df (validation_main_df)
print(f"Train Data Size: {len(train_x)}, Validation Data Size: {len(validation_x)}")
print(f"Train Data:")
print(f"  Buys List: {train_y.count(1)}")
print(f"  Don't Buy List: {train_y.count(0)}")
print(f"Validation Data:")
print(f"  Buys List: {validation_y.count(1)}")
print(f"  Don't Buys List: {validation_y.count(0)}")

model = Sequential ()

# Jupyter notebook LSTM is ok. Jupyter Lab LSTM is NG.
# We add three layers of LSTM
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences = True))
model.add(Dropout(0.2)) # we dropout 20% neurons.
model.add(BatchNormalization())

#model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1)) # we dropout 20% neurons.
model.add(BatchNormalization())

#model.add(LSTM(128, input_shape=(train_x.shape[1:])))
model.add(LSTM(128))
model.add(Dropout(0.2)) # we dropout 20% neurons.
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
          
model.add(Dense(2, activation="softmax"))

#opt = tf.keras.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss = 'sparse_categorical_crossentropy', 
               optimizer = opt,
               metrics = ['accuracy'])

## Add the tensorboard 
# trensorboard = TensorBoard(log_dir = f'logs/{NAME}')
tensorboard = TensorBoard (log_dir='log/{}'.format(NAME))
 
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session (config=tf.ConfigProto(gpu_options=gpu_options))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
sess = InteractiveSession(config=config)

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  # Unqiue_file name that will include the epoch and value
checkpoint = ModelCheckpoint ("models/{}.model".format(filepath, monitor="val_accuracy", 
                verbose =1, save_best_only=True, mode = 'max'))  # Save only the best ones.
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)
history = model.fit(train_x, train_y, batch_size = BATCH_SIZE, 
                epochs = EPOCHS, validation_data = (validation_x, validation_y),
                callbacks=[tensorboard, checkpoint])
#model.save("")

Train Data Size: 69188, Validation Data Size: 3062
Train Data:
  Buys List: 34594
  Don't Buy List: 34594
Validation Data:
  Buys List: 1531
  Don't Buys List: 1531


/home/peter/anaconda3/envs/tf/lib/python3.11/site-packages/tensorflow/python/client/session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1/10
1082/1082 [==============================] - ETA: 0s - loss: 0.7000 - accuracy: 0.5006INFO:tensorflow:Assets written to: models/RNN_Final-01-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-01-0.500.model/assets


1082/1082 [==============================] - 175s 156ms/step - loss: 0.7000 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.5020INFO:tensorflow:Assets written to: models/RNN_Final-02-0.500.model/assets


...

1082/1082 [==============================] - 158s 146ms/step - loss: 0.6934 - accuracy: 0.4990 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.4998INFO:tensorflow:Assets written to: models/RNN_Final-06-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-06-0.500.model/assets


1082/1082 [==============================] - 158s 146ms/step - loss: 0.6935 - accuracy: 0.4998 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.5007INFO:tensorflow:Assets written to: models/RNN_Final-07-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-07-0.500.model/assets


1082/1082 [==============================] - 158s 146ms/step - loss: 0.6935 - accuracy: 0.5007 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/10
 351/1082 [========>.....................] - ETA: 4:17 - loss: 0.6934 - accuracy: 0.4982

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
# https://stackoverflow.com/questions/47857437/keras-importerror-cannot-import-name-cudnnlstm
# tensorflow LSTM already has CuDNNLSTM
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"
# Deep Learning: Epoch = One Pass (Forward + Backward)
# Reinforcement Learning: Episode = Terminnal state (State, Reward)
EPOCHS = 10
BATCH_SIZE = 64     # 64 rows of dataset
# Uinque Model Name: Sequence_Length-Sequence_Period-Time_Stamp
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]]) 
    random.shuffle(sequential_data)
    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle (buys)
    random.shuffle (sells)
    # We want to know what is the minimum vlaue of this two lists.
    lower = min(len(buys), len(sells))
    # Upto 30K Buys
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)

    # We will do X, y for model.fit(X, y)
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
#print(main_df.head())  # how did we do??

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)

## here, split away some slice of the future data from the main main_df.
times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

model = Sequential ()

# Jupyter notebook LSTM is ok. Jupyter Lab LSTM is NG.
# We add three layers of LSTM
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences = True))
model.add(Dropout(0.2)) # we dropout 20% neurons.
model.add(BatchNormalization())

#model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1)) # we dropout 20% neurons.
model.add(BatchNormalization())

#model.add(LSTM(128, input_shape=(train_x.shape[1:])))
model.add(LSTM(128))
model.add(Dropout(0.2)) # we dropout 20% neurons.
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
          
model.add(Dense(2, activation="softmax"))

#opt = tf.keras.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss = 'sparse_categorical_crossentropy', 
               optimizer = opt,
               metrics = ['accuracy'])

## Add the tensorboard 
# trensorboard = TensorBoard(log_dir = f'logs/{NAME}')
tensorboard = TensorBoard (log_dir='log/{}'.format(NAME))
 
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session (config=tf.ConfigProto(gpu_options=gpu_options))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
sess = InteractiveSession(config=config)

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  # Unqiue_file name that will include the epoch and value
checkpoint = ModelCheckpoint ("models/{}.model".format(filepath, monitor="val_accuracy", 
                verbose =1, save_best_only=True, mode = 'max'))  # Save only the best ones.
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)
history = model.fit(train_x, train_y, batch_size = BATCH_SIZE, 
                epochs = EPOCHS, validation_data = (validation_x, validation_y),
                callbacks=[tensorboard, checkpoint])
#model.save("")

/tmp/ipykernel_63698/1141704108.py:88: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values


train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930
Epoch 1/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6990 - accuracy: 0.4984INFO:tensorflow:Assets written to: models/RNN_Final-01-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-01-0.500.model/assets


1218/1218 [==============================] - 166s 133ms/step - loss: 0.6990 - accuracy: 0.4984 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6934 - accuracy: 0.5014INFO:tensorflow:Assets written to: models/RNN_Final-02-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-02-0.500.model/assets


...

1218/1218 [==============================] - 177s 145ms/step - loss: 0.6934 - accuracy: 0.5006 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 9/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.4976INFO:tensorflow:Assets written to: models/RNN_Final-09-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-09-0.500.model/assets


1218/1218 [==============================] - 184s 151ms/step - loss: 0.6935 - accuracy: 0.4976 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 10/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.4984INFO:tensorflow:Assets written to: models/RNN_Final-10-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-10-0.500.model/assets


1218/1218 [==============================] - 159s 131ms/step - loss: 0.6935 - accuracy: 0.4984 - val_loss: 0.6932 - val_accuracy: 0.5000


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing

SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
# how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
RATIO_TO_PREDICT = "LTC-USD"
EPOCHS = 10  # how many passes through our data
BATCH_SIZE = 64  

NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
def classify(current, future):
    if float(future) > float(current):  
        # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0

def preprocess_df(df):
    #df = df.drop("future", 1)  # don't need this anymore.
    df = df.drop('future', axis=1)

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            # pct change "normalizes" the different currencies (each crypto coin has vastly 
            # diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = df[col].pct_change()  
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    # These will be our actual sequences. They are made with deque, 
    # which keeps the maximum length by popping out older values as new ones come in
    prev_days = deque(maxlen=SEQ_LEN)  


    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    # another shuffle, so the model doesn't get confused with all 1 class then the other.
    random.shuffle(sequential_data)  
   
    X = []
    y = []
    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
    return np.array(X), y  # return X and y...and make X a numpy array!

main_df = pd.DataFrame() # begin empty
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider

for ratio in ratios:  # begin iteration
    ratio = ratio.split('.csv')[0]  # split away the ticker from the file-name
    print(ratio)
    #dataset = f'training_datas/{ratio}.csv'  # get the full path to the file.
    dataset = f"crypto_data/{ratio}.csv"
    # read in specific file
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
#print(main_df.head())  # how did we do??

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)

## here, split away some slice of the future data from the main main_df.
times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

model = Sequential()
#model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

#model.add(CuDNNLSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

#model.add(CuDNNLSTM(128))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

#opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
        metrics=['accuracy']
)

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
# unique file name that will include the epoch and the validation acc for that epoch
filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', 
                                                      verbose=1, 
                                                      save_best_only=True, 
                                                      mode='max')) # saves only the best ones

# Train model
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

BTC-USD
LTC-USD
BCH-USD
ETH-USD


/tmp/ipykernel_63698/2791899738.py:109: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values


train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930


2023-10-23 19:26:49.058595: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-23 19:26:49.058629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: knode1
2023-10-23 19:26:49.058635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: knode1
2023-10-23 19:26:49.058789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.199.2
2023-10-23 19:26:49.058810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.199.2
2023-10-23 19:26:49.058815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.199.2


Epoch 1/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6993 - accuracy: 0.4980INFO:tensorflow:Assets written to: models/RNN_Final-01-0.500.model/assets


...

INFO:tensorflow:Assets written to: models/RNN_Final-09-0.500.model/assets


1218/1218 [==============================] - 151s 124ms/step - loss: 0.6935 - accuracy: 0.4979 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 10/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.4980INFO:tensorflow:Assets written to: models/RNN_Final-10-0.500.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-10-0.500.model/assets


1218/1218 [==============================] - 153s 126ms/step - loss: 0.6935 - accuracy: 0.4980 - val_loss: 0.6932 - val_accuracy: 0.5000
Test loss: 0.6931668519973755
Test accuracy: 0.5
INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1698114405/assets


INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1698114405/assets


In [6]:
# Previous case with initalized step 0.01 and decya = 0.9/10000 = 0.9E-4
#Retest with initalized rate = 0.001 and 
# decay = 1/1.E+6 step = 1.E-6
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing

SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
# how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
RATIO_TO_PREDICT = "LTC-USD"
EPOCHS = 10  # how many passes through our data
BATCH_SIZE = 64  

NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
def classify(current, future):
    if float(future) > float(current):  
        # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0

def preprocess_df(df):
    #df = df.drop("future", 1)  # don't need this anymore.
    df = df.drop('future', axis=1)

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            # pct change "normalizes" the different currencies (each crypto coin has vastly 
            # diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = df[col].pct_change()  
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    # These will be our actual sequences. They are made with deque, 
    # which keeps the maximum length by popping out older values as new ones come in
    prev_days = deque(maxlen=SEQ_LEN)  


    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    # another shuffle, so the model doesn't get confused with all 1 class then the other.
    random.shuffle(sequential_data)  
   
    X = []
    y = []
    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
    return np.array(X), y  # return X and y...and make X a numpy array!

main_df = pd.DataFrame() # begin empty
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider

for ratio in ratios:  # begin iteration
    ratio = ratio.split('.csv')[0]  # split away the ticker from the file-name
    print(ratio)
    #dataset = f'training_datas/{ratio}.csv'  # get the full path to the file.
    dataset = f"crypto_data/{ratio}.csv"
    # read in specific file
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
#print(main_df.head())  # how did we do??

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)

## here, split away some slice of the future data from the main main_df.
times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

model = Sequential()
#model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

#model.add(CuDNNLSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

#model.add(CuDNNLSTM(128))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

#opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000000,
    decay_rate=1.0)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
        metrics=['accuracy']
)

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
# unique file name that will include the epoch and the validation acc for that epoch
filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', 
                                                      verbose=1, 
                                                      save_best_only=True, 
                                                      mode='max')) # saves only the best ones

# Train model
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

BTC-USD
LTC-USD
BCH-USD
ETH-USD


/tmp/ipykernel_63698/2257097042.py:112: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values


train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930
Epoch 1/10


2023-10-23 20:37:36.081737: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 149610240 exceeds 10% of free system memory.


1218/1218 [==============================] - ETA: 0s - loss: 0.7175 - accuracy: 0.5188INFO:tensorflow:Assets written to: models/RNN_Final-01-0.557.model/assets


...

INFO:tensorflow:Assets written to: models/RNN_Final-09-0.570.model/assets


1218/1218 [==============================] - 150s 123ms/step - loss: 0.6662 - accuracy: 0.5935 - val_loss: 0.6822 - val_accuracy: 0.5702
Epoch 10/10
1218/1218 [==============================] - ETA: 0s - loss: 0.6602 - accuracy: 0.6037INFO:tensorflow:Assets written to: models/RNN_Final-10-0.579.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-10-0.579.model/assets


1218/1218 [==============================] - 150s 123ms/step - loss: 0.6602 - accuracy: 0.6037 - val_loss: 0.6799 - val_accuracy: 0.5793
Test loss: 0.6799372434616089
Test accuracy: 0.579274594783783
INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1698118650/assets


INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1698118650/assets
